In [ ]:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch

from dataclasses import dataclass
from typing import Dict, List, Union

from utils import SR, WRITE_ACCESS_TOKEN, clear_cache, change_pwd
change_pwd()

from huggingface_hub import login
login(WRITE_ACCESS_TOKEN)
from accelerate import Accelerator
from accelerate.logging import get_logger
from datasets import Audio, Dataset, load_dataset
from transformers import (
    Wav2Vec2Config,
    Wav2Vec2Model,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2Processor,
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import argparse
import math
import os
import pandas as pd
import numpy as np
import soundfile as sf

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Union
from tqdm import tqdm

import datasets
import torch
from torch.utils.data import Dataset
from datasets import DatasetDict, concatenate_datasets, load_dataset, IterableDatasetDict
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from huggingface_hub import Repository
from transformers import (
    SchedulerType,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
    get_scheduler,
    set_seed,
)
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers.utils import get_full_repo_name
import time

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, files, sep, sr, audio_column_name, duration_column_name, min_duration, max_duration, dataset_name):
        self.sep = sep
        self.sr = sr
        self.min_duration = min_duration
        self.max_duration = max_duration
        self.audio_column_name = audio_column_name
        self.duration_column_name = duration_column_name
        self.dataset_name = dataset_name
        self.data = self.load_ds(files)

    def load_ds(self, all_files):
        li = []
        for filename in all_files:

            df = pd.read_csv(filename, sep=self.sep, engine="python")
            li.append(df)
        data = pd.concat(li, axis=0, ignore_index=True)

        if self.duration_column_name in data.columns:
            data = data[data[self.duration_column_name] >= self.min_duration]
            print("Mean duration: ", data[self.duration_column_name].mean())
        return data

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        batch = {}
        batch["input_values"] = sf.read(os.path.join(self.dataset_name, item["path"]))[0]

        if len(batch["input_values"])//self.sr > self.max_duration:
            start = np.random.randint(0, len(batch["input_values"]) - self.max_duration * self.sr)
            batch["input_values"] = batch["input_values"][start : start + int(self.max_duration * self.sr)]

        return batch


@dataclass
class DataCollatorForWav2Vec2Pretraining:
    """
    Data collator that will dynamically pad the inputs received and prepare masked indices
    for self-supervised pretraining.

    Args:
        model (:class:`~transformers.Wav2Vec2ForPreTraining`):
            The Wav2Vec2 model used for pretraining. The data collator needs to have access
            to config and ``_get_feat_extract_output_lengths`` function for correct padding.
        feature_extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`):
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    model: Wav2Vec2ForPreTraining
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = "longest"
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # reformat list to dict and set to pytorch format
        batch = self.feature_extractor.pad(
            features,
            padding=self.padding,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        device = batch["input_values"].device
        batch_size = batch["input_values"].shape[0]

        mask_indices_seq_length = self.model._get_feat_extract_output_lengths(batch["input_values"].shape[-1])
        # make sure masked sequence length is a Python scalar
        mask_indices_seq_length = int(mask_indices_seq_length)

        # make sure that no loss is computed on padded inputs
        if batch.get("attention_mask") is not None:
            # compute real output lengths according to convolution formula
            batch["sub_attention_mask"] = self.model._get_feature_vector_attention_mask(
                mask_indices_seq_length, batch["attention_mask"]
            )

        features_shape = (batch_size, mask_indices_seq_length)

        # sample randomly masked indices
        mask_time_indices = _compute_mask_indices(
            features_shape,
            self.model.config.mask_time_prob,
            self.model.config.mask_time_length,
            attention_mask=batch.get("sub_attention_mask")
        )
        # sample negative indices
        sampled_negative_indices = _sample_negative_indices(
            features_shape,
            self.model.config.num_negatives,
            mask_time_indices=mask_time_indices,
        )
        batch["mask_time_indices"] = torch.tensor(mask_time_indices, dtype=torch.long, device=device)
        batch["sampled_negative_indices"] = torch.tensor(sampled_negative_indices, dtype=torch.long, device=device)

        return batch


def multiply_grads(params, c):
    """Multiplies grads by a constant *c*."""
    for p in params:
        if p.grad is not None:
            if torch.is_tensor(c):
                c = c.to(p.grad.device)
            p.grad.data.mul_(c)


def get_grad_norm(params, scale=1):
    """Compute grad norm given a gradient scale."""
    total_norm = 0.0
    for p in params:
        if p.grad is not None:
            param_norm = (p.grad.detach().data / scale).norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm**0.5
    return total_norm

In [ ]:
from load_nchlt_2 import load_nchlt_2

load_nchlt_2(dataset_name="nchlt_afr")

100%|██████████| 4.89G/4.89G [26:47<00:00, 3.04MiB/s]


File nchlt_afr.tar.gz downloaded successfully!



100%|██████████| 5.06G/5.06G [20:21<00:00, 4.14MiB/s]


File nchlt_xho.tar.gz downloaded successfully!



100%|██████████| 103142/103142 [00:00<00:00, 107504.91it/s]


OSError: ignored

In [ ]:
logger = get_logger(__name__)

push_to_hub = True
user_name = "lucas-meyer"
repo_name = "wav2vec2-from-scratch-nchlt_afr"
resume = False

train_datasets = [os.path.join("nchlt_afr", "train.csv")]
val_datasets = [os.path.join("nchlt_afr", "validation.csv")]
separator = ","
audio_column_name = "path"
duration_column_name = "duration"
min_duration_in_seconds = 0.5
max_duration_in_seconds = 25.0

per_device_train_batch_size = 16
per_device_eval_batch_size = 8
gradient_accumulation_steps = 8
learning_rate = 0.0001
lr_scheduler_type = "linear"
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 1e-8
weight_decay = 0.01

max_gumbel_temperature = 2.0
min_gumbel_temperature = 0.5
gumbel_temperature_decay = 0.999995

num_train_epochs = 20
num_warmup_steps = 32000
max_train_steps = 200000
saving_steps = 100
logging_steps = 1

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator(dispatch_batches=False)
logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    # set up tensorboard if available
    writer = SummaryWriter(repo_name + '/logs', max_queue=5, flush_secs=30)

# Set the training seed now.
set_seed(42)

# Handle the repository creation
if accelerator.is_main_process:
    repo = Repository(repo_name, clone_from=f"{user_name}/{repo_name}")
    os.makedirs(repo_name, exist_ok=True)
    print("poes ya")

accelerator.wait_for_everyone()

# Download data
train_dataset = CustomDataset(
    files=train_datasets,
    sep=separator,
    audio_column_name=audio_column_name,
    duration_column_name=duration_column_name,
    sr=SR,
    min_duration=min_duration_in_seconds,
    max_duration=max_duration_in_seconds,
    dataset_name="nchlt_afr",
)

val_dataset = CustomDataset(
    files=val_datasets,
    sep=separator,
    audio_column_name=audio_column_name,
    duration_column_name=duration_column_name,
    sr=SR,
    min_duration=min_duration_in_seconds,
    max_duration=max_duration_in_seconds,
    dataset_name="nchlt_afr",
)

# Load feature_extractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")
# only normalized-inputs-training is supported
if not feature_extractor.do_normalize:
    raise ValueError(
        "Training is only supported for normalized inputs. Make sure ``feature_extractor.do_normalize == True``"
    )

# Load model config
config = Wav2Vec2Config()
# if not config.do_stable_layer_norm or config.feat_extract_norm != "layer":
#     raise ValueError(
#         "PreTraining is only supported for ``config.do_stable_layer_norm=True`` and"
#         " ``config.feat_extract_norm='layer'"
#     )


# initialize random model
model = Wav2Vec2ForPreTraining(config)
# if load_from_pretrained is not None:
#     try:
#         model = model.from_pretrained(model_name_or_path)
#     except:
#         print("!!!!! Warning: Pretrained model may not exist. Start training from Scratch")

# Activate gradient checkpointing
model.gradient_checkpointing_enable()

# Define data collator, optimizer and scheduler
data_collator = DataCollatorForWav2Vec2Pretraining(
    model=model, feature_extractor=feature_extractor, pad_to_multiple_of=None
)

train_dataloader = DataLoader(
    train_dataset,
    collate_fn=data_collator,
    batch_size=per_device_train_batch_size,
    shuffle=True,
    num_workers=16,
    pin_memory=True,
    prefetch_factor=16
)

eval_dataloader = DataLoader(
    val_dataset,
    collate_fn=data_collator,
    batch_size=per_device_eval_batch_size,
    num_workers=16
)

# Optimizer
optimizer =  torch.optim.AdamW(
    list(model.parameters()),
    lr=learning_rate,
    betas=[adam_beta1, adam_beta2],
    eps=adam_epsilon,
)

lr_scheduler = get_scheduler(
    name=lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=max_train_steps,
)

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)
if resume:
    print("******Resume checkpoint******")
    accelerator.load_state(repo_name)
    checkpoint = torch.load(os.path.join(repo_name, 'latest_checkpoint.pt'),
                            map_location="cpu")


 # Train
total_batch_size = per_device_train_batch_size * accelerator.num_processes * gradient_accumulation_steps

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)


if max_train_steps is None:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch


# Afterwards we recalculate our number of training epochs
num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

if accelerator.is_main_process:
    print("Number of training data: ", len(train_dataset))
    print("total_batch_size: ", total_batch_size)
    print("num_update_steps_per_epoch: ", num_update_steps_per_epoch)
    print("num_train_epochs: ", num_train_epochs)

# Only show the progress bar once on each machine.
completed_steps = checkpoint['completed_steps'] + 1 if resume else 0
starting_epoch = checkpoint['epoch'] if resume else 0
progress_bar = tqdm(initial = completed_steps, total = max_train_steps, disable=not accelerator.is_local_main_process)

print(f"******STARTING AT EPOCH {starting_epoch} - STEP {completed_steps}******")


for epoch in range(starting_epoch, num_train_epochs):
    if accelerator.is_main_process:
        print(f"\nEpoch {epoch}: ")
    model.train()
    for step, batch in enumerate(train_dataloader):
        # compute num of losses
        num_losses = batch["mask_time_indices"].sum()
        sub_attention_mask = batch.pop("sub_attention_mask", None)
        sub_attention_mask = (
            sub_attention_mask if sub_attention_mask is not None else torch.ones_like(batch["mask_time_indices"])
        )
        percent_masked = num_losses / sub_attention_mask.sum()

        # forward
        outputs = model(**batch)

        # divide loss by gradient accumulation steps since gradients
        # are accumulated for multiple backward passes in PyTorch
        loss = outputs.loss / gradient_accumulation_steps
        accelerator.backward(loss)

        # make sure that `num_losses` is summed for distributed training
        # and average gradients over losses of all devices
        if accelerator.state.num_processes > 1:
            num_losses = accelerator.gather(num_losses).sum()
            gradient_multiplier = accelerator.state.num_processes / num_losses
            multiply_grads(model.module.parameters(), gradient_multiplier)
        else:
            multiply_grads(model.parameters(), 1 / num_losses)

        # update step
        if (step + 1) % gradient_accumulation_steps == 0:

            # compute grad norm for monitoring
            scale = (
                accelerator.scaler._scale.item()
                if hasattr(accelerator, "scaler") and accelerator.scaler is not None
                else 1
            )
            if accelerator.state.num_processes > 1:
                grad_norm = get_grad_norm(model.module.parameters(), scale)
            else:
                grad_norm = get_grad_norm(model.parameters(), scale)

            # update parameters
            optimizer.step()
            optimizer.zero_grad()

            if not accelerator.optimizer_step_was_skipped:
                lr_scheduler.step()
            elif accelerator.is_local_main_process:
                progress_bar.write(
                    f"Gradients have overflown - skipping update step... Updating gradient scale to {scale}..."
                )

            # update gumbel temperature
            gumbel_temperature = max(
                max_gumbel_temperature * gumbel_temperature_decay**completed_steps,
                min_gumbel_temperature,
            )
            if hasattr(model, "module"):
                model.module.set_gumbel_temperature(gumbel_temperature)
            else:
                model.set_gumbel_temperature(gumbel_temperature)

            progress_bar.update(1)
            completed_steps += 1

        # Log all results
        if (step + 1) % (gradient_accumulation_steps * logging_steps) == 0:
            loss.detach()
            outputs.contrastive_loss.detach()
            outputs.diversity_loss.detach()
            cosine_sim = torch.cosine_similarity(outputs.projected_states, outputs.projected_quantized_states, dim=-1)
            cosine_sim = cosine_sim[batch["mask_time_indices"].to(torch.bool)].mean()

            if accelerator.state.num_processes > 1:
                loss = accelerator.gather(loss).sum()
                outputs.contrastive_loss = accelerator.gather(outputs.contrastive_loss).sum()
                outputs.diversity_loss = accelerator.gather(outputs.diversity_loss).sum()
                percent_masked = accelerator.gather(percent_masked).sum()
                cosine_sim = accelerator.gather(cosine_sim).mean()

            train_logs = {
                "step": torch.tensor((step + 1) // gradient_accumulation_steps, dtype=torch.int32),
                "loss": (loss * gradient_accumulation_steps) / num_losses,
                "contrast_loss": outputs.contrastive_loss / num_losses,
                "div_loss": outputs.diversity_loss / num_losses,
                "%_mask_idx": percent_masked / accelerator.num_processes,
                "ppl": outputs.codevector_perplexity,
                "lr": torch.tensor(lr_scheduler.get_lr()),
                "temp": torch.tensor(gumbel_temperature),
                "grad_norm": torch.tensor(grad_norm),
                "cosine_sim": cosine_sim * 100
            }
            log_str = ""
            for k, v in train_logs.items():
                log_str += "| {}: {:.3e}".format(k, v.item())

            if accelerator.is_local_main_process:
                progress_bar.write(log_str)
                for k, v in train_logs.items():
                    writer.add_scalar('TRAIN' + '/' + k, v, completed_steps)


        # save model every `saving_steps` steps
        if (step + 1) % (gradient_accumulation_steps * saving_steps) == 0:
            if (push_to_hub and epoch < num_train_epochs - 1) or repo_name is not None:
                accelerator.wait_for_everyone()
                unwrapped_model = accelerator.unwrap_model(model)
                unwrapped_model.save_pretrained(
                        repo_name + f'/saved_model/epoch_{epoch}', is_main_process=accelerator.is_main_process, save_function=accelerator.save
                    )
                if accelerator.is_main_process:
                    feature_extractor.save_pretrained(repo_name + f'/saved_model/epoch_{epoch}')
                    print("****Saving checkpoint*****")
                    state_dict = {
                        "completed_steps": completed_steps,
                        "epoch": epoch
                    }
                    torch.save(state_dict, os.path.join(repo_name, "latest_checkpoint.pt"))
                accelerator.save_state(repo_name)

            if (push_to_hub and epoch < num_train_epochs - 1) and accelerator.is_main_process:
                repo.push_to_hub(
                    commit_message=f"Training in progress step {completed_steps}",
                    blocking=False,
                    auto_lfs_prune=True,
                )

        # if completed steps > `max_train_steps` stop
        if completed_steps >= max_train_steps:
            break

    print("******END OF EPOCH******")
    # Validate!
    model.eval()

    # init logs
    val_logs = {
        "val_loss": 0,
        "val_contrastive_loss": 0,
        "val_diversity_loss": 0,
        "val_num_losses": 0,
    }
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch.pop("sub_attention_mask", None)
            outputs = model(**batch)

        val_logs["val_loss"] += outputs.loss
        val_logs["val_contrastive_loss"] += outputs.contrastive_loss
        val_logs["val_diversity_loss"] += outputs.diversity_loss
        val_logs["val_num_losses"] += batch["mask_time_indices"].sum()

    # sum over devices in multi-processing
    if accelerator.num_processes > 1:
        val_logs = {k: accelerator.gather(v).sum() for k, v in val_logs.items()}

    val_logs = {k: v / val_logs["val_num_losses"] for k, v in val_logs.items()}

    log_str = ""
    for k, v in val_logs.items():
        log_str += "| {}: {:.3e}".format(k, v.item())

    if accelerator.is_local_main_process:
        progress_bar.write(log_str)
        for k, v in val_logs.items():
            writer.add_scalar('VALIDATION' + '/' + k, v, epoch)

    if repo_name is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(
                repo_name + f'/saved_model/epoch_{epoch}', is_main_process=accelerator.is_main_process, save_function=accelerator.save
            )
        if accelerator.is_main_process:
            feature_extractor.save_pretrained(repo_name + f'/saved_model/epoch_{epoch}')
            print("****Saving checkpoint*****")
            state_dict = {
                "completed_steps": completed_steps,
                "epoch": epoch
            }
            torch.save(state_dict, os.path.join(repo_name, "latest_checkpoint.pt"))

        accelerator.save_state(repo_name)
        if accelerator.is_main_process:
            if push_to_hub:
                repo.push_to_hub(commit_message="End of training", auto_lfs_prune=True)


OSError: ignored